In [ ]:
# ======== COMMON CONFIG (run once) ==========================================
import os, sys, re, warnings, argparse, pandas as pd
warnings.filterwarnings("ignore")

from rolling_framework import Machine           # <-- 프로젝트의 핵심 API

# ---- 파일 경로 --------------------------------------------------------------
DATA_DIR      = "data/"
Y_FILE        = os.path.join(DATA_DIR, "exrets.csv")
SLOPE_FILE    = os.path.join(DATA_DIR, "slope.csv")
YL_FILE       = os.path.join(DATA_DIR, "yl_all.csv")
MACRO_FILE    = os.path.join(DATA_DIR, "MacroFactors.csv")
IV_FILE       = os.path.join(DATA_DIR, "imp_vol.csv")
RV_FILE       = os.path.join(DATA_DIR, "real_vol.csv")

OUT_DIR       = "./output";  os.makedirs(OUT_DIR, exist_ok=True)

# ---- 샘플/예측 구간 ---------------------------------------------------------
BURN_START, BURN_END   = "199009", "200609"
PERIOD_START, PERIOD_END = "199009", "202312"
HORIZON = 12                           # months ahead

MATURITIES = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

# ---- 유틸 함수 --------------------------------------------------------------
def _load_csv(path, name):
    try:  return pd.read_csv(path, index_col="Time")
    except FileNotFoundError as e:
        sys.exit(f"[ERROR] missing {name} → {e.filename}")

def _align_time(*dfs):
    idx=None
    for d in dfs: idx = d.index if idx is None else idx.intersection(d.index)
    return [d.loc[idx].sort_index() for d in dfs]

def _direct_pairs(slope_cols, y_cols):
    mk = lambda s: re.search(r"(\d+)", s).group(1) if re.search(r"(\d+)", s) else None
    y_map = {mk(c): c for c in y_cols}
    return [(sc, y_map[mk(sc)]) for sc in slope_cols if mk(sc) in y_map]

# ---- 데이터 로드 & 정렬 ------------------------------------------------------
y     = _load_csv(Y_FILE,   "exrets")
slope = _load_csv(SLOPE_FILE, "slope")
yl    = _load_csv(YL_FILE,   "yl_all")
macro = _load_csv(MACRO_FILE,"MacroFactors")
iv    = _load_csv(IV_FILE,   "imp_vol")
rv    = _load_csv(RV_FILE,   "real_vol")


# 타깃 열 필터
y_cols = [c for c in MATURITIES if c in y.columns]
if not y_cols: sys.exit("[ERROR] MATURITIES not in exrets")
y = y[y_cols]

# 시간축 맞추기
y, slope, yl, macro, iv, rv = _align_time(y, slope, yl, macro, iv, rv)

# slope->y 자동 매핑  ex) slope_2 -> xr_2
DIRECT_PAIRS = _direct_pairs(slope.columns, y_cols)

print("✓ Loaded data shapes:",
      {k:v.shape for k,v in [("y",y),("slope",slope),("yl",yl),("macro",macro),("iv",iv),("rv",rv)]})
print("✓ direct map pairs :", DIRECT_PAIRS)

DNN_DUAL rolling:   4%|▍         | 20/520 [03:47<1:34:56, 11.39s/it]


KeyboardInterrupt: 

In [ ]:
# ===== CASE 3 : iv(branch1) + rv(branch2)  (no direct) ===============
X3 = pd.concat([iv, rv], axis=1)
opt3 = {
    "branches":[
        {"cols":rv.columns.tolist(),   "hidden":(3,), "drop":0.1},
        {"cols":iv.columns.tolist(),"hidden":(3,), "drop":0.1},
    ],
    "direct_map":[],                   # direct 없음
    "head_hidden":3,
}
grid3 = {
    "dnn__optimizer__lr":[1e-3],
    "dnn__optimizer__weight_decay":[1e-4],
    "dnn__lr_br":[[1e-3,1e-3]],        # 2-branch
    "dnn__lr_head":[1e-3],
    "dnn__module__head_hidden":[3],
}
m3 = Machine(X3, y, "DNN_NBR", option=opt3, params_grid=grid3   ,
             burn_in_start=BURN_START, burn_in_end=BURN_END,
             period=[PERIOD_START, PERIOD_END], forecast_horizon=HORIZON)
print("\n▶ CASE-3  imp_vol(branch1) + rv(branch2)  (no direct)")
m3.training()
print(m3.R2OOS())
print(m3.MSEOOS())